<a href="https://colab.research.google.com/github/nandhukumar86/mykagglesubmissions/blob/master/house_prices_advanced_regression_techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install kaggle
! pip install -q kaggle

# upload file to Colab
from google.colab import files
files.upload()

# create kaggle folder in root and copy the file.
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

# provide permissions
! chmod 600 ~/.kaggle/kaggle.json

# download competition files by providing competition name
! kaggle competitions download -c 'house-prices-advanced-regression-techniques'

# unzip the files and folders
#! unzip train.csv.zip -d train
#! unzip test.csv.zip -d test


Saving kaggle.json to kaggle.json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
  0% 0.00/450k [00:00<?, ?B/s]
100% 450k/450k [00:00<00:00, 66.7MB/s]
  0% 0.00/441k [00:00<?, ?B/s]
100% 441k/441k [00:00<00:00, 61.7MB/s]
  0% 0.00/13.1k [00:00<?, ?B/s]
100% 13.1k/13.1k [00:00<00:00, 13.5MB/s]
  0% 0.00/31.2k [00:00<?, ?B/s]
100% 31.2k/31.2k [00:00<00:00, 33.9MB/s]


#Basic Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)
%matplotlib inline


# Load Data

In [ ]:
df_train = pd.read_csv('/content/train.csv')
df_test = pd.read_csv('/content/test.csv')

In [ ]:
df_train.columns, df_test.columns

(Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
        'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
        'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
        'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
        'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
        'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
        'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
        'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
        'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
        'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
        'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
        'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
        'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
        'GarageCond

# Removing Id columns.

In [ ]:
y_id = df_test.pop('Id')

In [ ]:
df_train.drop(columns= ['Id'], inplace=True)

In [ ]:
y_train = df_train.pop('SalePrice')

In [ ]:
all(df_train.columns == df_test.columns)

True

In [ ]:
X_features = df_train.columns

In [ ]:
X_train = df_train.copy(deep = True)
X_test = df_test.copy(deep = True)

In [ ]:
X_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [ ]:
y_train.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [ ]:
X_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [ ]:
categorical_columns = X_train.describe(include = ['O']).columns
numerical_columns = X_train.describe().columns

In [ ]:
categorical_columns.size + numerical_columns.size == X_features.size

True

## Removing highly correlated features

In [ ]:
corr_matrix = X_train[numerical_columns].corr()
            
upper = corr_matrix.abs().where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(np.bool))
high_correlation_features = [column for column in upper.columns if any(upper[column] > 0.8)]


In [ ]:
high_correlation_features

['1stFlrSF', 'TotRmsAbvGrd', 'GarageYrBlt', 'GarageArea']

In [ ]:
X_train.drop(columns= high_correlation_features, inplace=True)
X_test.drop(columns= high_correlation_features, inplace=True)

In [ ]:
X_features = X_train.columns

In [ ]:
numerical_columns = numerical_columns.drop(high_correlation_features)

In [ ]:
numerical_columns.size + categorical_columns.size == X_features.size

True

# Exploratory Data Analysis

Checking for columns more than 50% null

In [ ]:
temp1 = pd.DataFrame(X_train.isnull().sum())
temp1.columns = ['Count']


In [ ]:
X_train.shape

(1460, 75)

In [ ]:
temp1.query('Count > 750') #approximately more than 50% of test data

,Count
Alley,1369
PoolQC,1453
Fence,1179
MiscFeature,1406


The above columns has more null values and it does not store good amount of data to train the model. Hence Dropping these columns.

### Removing high NA features which constitutes 50% of NA

In [ ]:
high_na_featues = temp1.query('Count > 750').index

In [ ]:
high_na_featues

Index(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')

In [ ]:
X_train.drop(columns= high_na_featues, inplace=True)
X_test.drop(columns= high_na_featues, inplace=True)

In [ ]:
categorical_columns = X_train.describe(include = ['O']).columns
numerical_columns = X_train.describe().columns

In [ ]:
X_train.columns.size, X_test.columns.size

(71, 71)

In [ ]:
categorical_columns.size +  numerical_columns.size

71

In [ ]:
cat_column_counts = pd.DataFrame(X_train.describe(include=['O']).T)
cat_column_counts.columns = ['count', 'unique', 'top', 'freq']

### Removing Categorical columns with which top category occupies 90% of the data

In [ ]:
high_freq_columns = cat_column_counts.query('freq > 1300').index

In [ ]:
X_train.drop(columns= high_freq_columns, inplace=True)
X_test.drop(columns= high_freq_columns, inplace=True)

In [ ]:
categorical_columns = X_train.describe(include = ['O']).columns
numerical_columns = X_train.describe().columns

In [ ]:
X_train.columns.size, X_test.columns.size

(57, 57)

In [ ]:
categorical_columns.size +  numerical_columns.size

57

In [ ]:
X_train.describe().T

,count,mean,std,min,25%,50%,75%,max
MSSubClass,1460.0,56.897260,42.300571,20.0,20.00,50.0,70.00,190.0
LotFrontage,1201.0,70.049958,24.284752,21.0,59.00,69.0,80.00,313.0
LotArea,1460.0,10516.828082,9981.264932,1300.0,7553.50,9478.5,11601.50,215245.0
OverallQual,1460.0,6.099315,1.382997,1.0,5.00,6.0,7.00,10.0
OverallCond,1460.0,5.575342,1.112799,1.0,5.00,5.0,6.00,9.0
YearBuilt,1460.0,1971.267808,30.202904,1872.0,1954.00,1973.0,2000.00,2010.0
YearRemodAdd,1460.0,1984.865753,20.645407,1950.0,1967.00,1994.0,2004.00,2010.0
MasVnrArea,1452.0,103.685262,181.066207,0.0,0.00,0.0,166.00,1600.0
BsmtFinSF1,1460.0,443.639726,456.098091,0.0,0.00,383.5,712.25,5644.0
BsmtFinSF2,1460.0,46.549315,161.319273,0.0,0.00,0.0,0.00,1474.0


### Removing columns with more than 50% zeros

In [ ]:
zero_features = pd.DataFrame(X_train[X_train == 0].count())
zero_features.columns = ['count']

In [ ]:
high_0_features = zero_features.query('count > 750').index

In [ ]:
X_train.drop(columns= high_0_features, inplace=True)
X_test.drop(columns= high_0_features, inplace=True)

In [ ]:
categorical_columns = X_train.describe(include = ['O']).columns
numerical_columns = X_train.describe().columns

In [ ]:
X_train.columns.size, X_test.columns.size

(44, 44)

In [ ]:
categorical_columns.size +  numerical_columns.size

44

In [ ]:
# sns.pairplot(X_train[numerical_columns], corner=True)

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import PCA

pl_n = Pipeline([
                 ('imputerNaN', SimpleImputer(missing_values=np.nan, strategy='median')),
                 ('imputer0', SimpleImputer(missing_values=0, strategy='median')),
                 ('std', StandardScaler()),
                 ('power', PowerTransformer())
])

pl_c = Pipeline([
                 ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
                 ('std', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([
                        ('num', pl_n, numerical_columns),
                        ('cat', pl_c, categorical_columns)
])

# Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

pl = Pipeline([
               ('ct', ct),
               ('md', LinearRegression())
])

space = dict()
space['md__fit_intercept'] = [True, False]
space['md__normalize'] = [True, False]

model = GridSearchCV(pl, param_grid = space, cv = 5)

model.fit(X_train, np.log(y_train))

print('Linear Regression')

print('Training Score: ', model.score(X_train, np.log(y_train)))
print('Validation Score:', model.best_score_)

# print(model)


Linear Regression
Training Score:  0.9211997742593504
Validation Score: 0.8790935481661564


# Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures

pl = Pipeline([
               ('ct', ct),
               ('lr', Ridge())
])

space = dict()
space['lr__fit_intercept'] = [True, False]
space['lr__normalize'] = [True, False]
space['lr__alpha'] = np.linspace(0.01, 1,10)

model = RandomizedSearchCV(pl, param_distributions = space, cv = 5)

model.fit(X_train, np.log(y_train))

print('Ridge Regression')

print('Training Score: ', model.score(X_train, np.log(y_train)))
print('Validation Score:', model.best_score_)


Ridge Regression
Training Score:  0.9935983903043111
Validation Score: 0.881409336825602


# Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso

pl = Pipeline([
               ('ct', ct),
               ('lr', Lasso())
])

space = dict()
space['lr__fit_intercept'] = [True, False]
space['lr__normalize'] = [True, False]
# space['lr__alpha'] = [0.75, 1]

model = GridSearchCV(pl, param_grid = space, cv = 5)

model.fit(X_train, np.log(y_train))

print('Lasso Regression')

print('Training Score: ', model.score(X_train, np.log(y_train)))
print('Validation Score:', model.best_score_)


Lasso Regression
Training Score:  0.0
Validation Score: -0.003468324498284092


# Elastic Net

In [ ]:
from sklearn.linear_model import ElasticNet

pl = Pipeline([
               ('ct', ct),
               ('lr', ElasticNet())
])

space = dict()
space['lr__fit_intercept'] = [True, False]
space['lr__normalize'] = [True, False]
space['lr__alpha'] = [0.25,0.5,0.75,1]

model = GridSearchCV(pl, param_grid = space, cv = 5)

model.fit(X_train, np.log(y_train))

print('ElasticNet Regression')

print('Training Score: ', model.score(X_train, np.log(y_train)))
print('Validation Score:', model.best_score_)


ElasticNet Regression
Training Score:  0.6244758167417034
Validation Score: 0.6228321538637032


In [ ]:
!pip install xgboost

# XGBoost Regressor

In [ ]:
! pip install xgboost

In [ ]:
from xgboost import XGBRegressor

pl = Pipeline([
               ('ct', ct),
               ('lr', XGBRegressor())
])

space = dict()
space['lr__n_estimators'] = [50,100,150]
space['lr__max_depth'] = [5, 10, 15]
space['lr__learning_rate'] = [0.1, 0.2, 0.3]
space['lr__base_score'] = [0.1,0.25,0.5]


model = RandomizedSearchCV(pl, param_distributions=space, cv = 5)

model.fit(X_train, np.log(y_train))

print('XGBoost Regression')

print('Training Score: ', model.score(X_train, np.log(y_train)))
print('Validation Score:', model.best_score_)

[07:46:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:46:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:46:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:46:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:46:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:46:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:46:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:46:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

# CatBoost Regressor

In [116]:
! pip install catboost

     |████████████████████████████████| 66.3MB 56kB/s 


In [139]:
from catboost import CatBoostRegressor

pl = Pipeline([
               ('ct', ct),
               ('lr', CatBoostRegressor(verbose=False))
])

space = dict()
space['lr__learning_rate'] = np.linspace(0.005, 0.15, 10)
space['lr__n_estimators'] = [500, 600, 700, 800, 900, 1000]


model = RandomizedSearchCV(pl, param_distributions=space, cv = 5)

model.fit(X_train, np.log(y_train))

print('CatBoost Regression')

print('Training Score: ', model.score(X_train, np.log(y_train)))
print('Validation Score:', model.best_score_)



/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: 'float' object cannot be interpreted as an integer

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: 'float' object cannot be interpreted as an integer

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: 'float' object cannot be interpreted as an integer

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator f

TypeError: ignored

In [135]:
catBModel = model
model.best_estimator_

Pipeline(memory=None,
         steps=[('ct',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputerNaN',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                                    

Based on the above analysis, Ridge performs better.

In [126]:
from sklearn.linear_model import Ridge

pl = Pipeline([
               ('ct', ct),
               ('lr', Ridge())
])

space = dict()
space['lr__fit_intercept'] = [True, False]
space['lr__normalize'] = [True, False]
space['lr__alpha'] = np.linspace(0.1, 1,20)

model = RandomizedSearchCV(pl, param_distributions = space, cv = 5)

model.fit(X_train, np.log(y_train))

print('Ridge Regression')

print('Training Score: ', model.score(X_train, np.log(y_train)))
print('Validation Score:', model.best_score_)

ridgeModel = model.best_estimator_

Ridge Regression
Training Score:  0.9207216160653777
Validation Score: 0.8838460446114093


In [127]:
from sklearn.linear_model import LinearRegression

pl = Pipeline([
               ('ct', ct),
               ('md', LinearRegression())
])

space = dict()
space['md__fit_intercept'] = [True, False]
space['md__normalize'] = [True, False]

model = GridSearchCV(pl, param_grid = space, cv = 5)

model.fit(X_train, np.log(y_train))

print('Linear Regression')

print('Training Score: ', model.score(X_train, np.log(y_train)))
print('Validation Score:', model.best_score_)

# print(model)
linRegModel = model.best_estimator_


Linear Regression
Training Score:  0.9211997742593504
Validation Score: 0.8790935481661564


In [ ]:
from xgboost import XGBRegressor

pl = Pipeline([
               ('ct', ct),
               ('lr', XGBRegressor())
])

space = dict()
space['lr__n_estimators'] = [50,100,150]
space['lr__max_depth'] = [5, 10, 15]
space['lr__learning_rate'] = [0.1, 0.2, 0.3]
space['lr__base_score'] = [0.1,0.25,0.5]


model = RandomizedSearchCV(pl, param_distributions=space, cv = 5)

model.fit(X_train, np.log(y_train))

print('XGBoost Regression')

print('Training Score: ', model.score(X_train, np.log(y_train)))
print('Validation Score:', model.best_score_)

xgbModel = model

[08:09:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

# Stacking Regressor

In [ ]:
from sklearn.ensemble import StackingRegressor, RandomForestRegressor


model = StackingRegressor(
    estimators = [
                  ('linreg',linRegModel),
                  ('ridge',ridgeModel)
    ],
    passthrough = False,
    final_estimator=RandomForestRegressor(n_estimators=10)
)

model.fit(X_train, np.log(y_train))

print('Stacking Regression')

print('Training Score: ', model.score(X_train, np.log(y_train)))

from sklearn.model_selection import cross_val_score

scores = cross_val_score(model,X_train, np.log(y_train), cv = 5)
print('Cross Val Score: ', scores.mean())

Stacking Regression
Training Score:  0.8897854229697493
Cross Val Score:  0.8556959703204043


# Voting Regressor

In [128]:
from sklearn.ensemble import VotingRegressor

model = VotingRegressor([
                         ('linreg',linRegModel),
                         ('cat',catBModel),
                         ('ridge',ridgeModel)
])

model.fit(X_train, np.log(y_train))

print('Voting Regression')

print('Training Score: ', model.score(X_train, np.log(y_train)))

from sklearn.model_selection import cross_val_score

scores = cross_val_score(model,X_train, np.log(y_train), cv = 5)
print('Cross Val Score: ', scores.mean())


Streaming output truncated to the last 5000 lines.
501:	learn: 0.0122302	total: 1.7s	remaining: 1.35s
502:	learn: 0.0121966	total: 1.71s	remaining: 1.35s
503:	learn: 0.0121403	total: 1.71s	remaining: 1.34s
504:	learn: 0.0121234	total: 1.71s	remaining: 1.34s
505:	learn: 0.0120620	total: 1.72s	remaining: 1.34s
506:	learn: 0.0120389	total: 1.72s	remaining: 1.33s
507:	learn: 0.0119942	total: 1.72s	remaining: 1.33s
508:	learn: 0.0119420	total: 1.73s	remaining: 1.33s
509:	learn: 0.0119187	total: 1.73s	remaining: 1.32s
510:	learn: 0.0118697	total: 1.73s	remaining: 1.32s
511:	learn: 0.0118332	total: 1.74s	remaining: 1.31s
512:	learn: 0.0118065	total: 1.74s	remaining: 1.31s
513:	learn: 0.0117692	total: 1.74s	remaining: 1.31s
514:	learn: 0.0117228	total: 1.75s	remaining: 1.3s
515:	learn: 0.0116847	total: 1.75s	remaining: 1.3s
516:	learn: 0.0116487	total: 1.75s	remaining: 1.3s
517:	learn: 0.0115954	total: 1.75s	remaining: 1.29s
518:	learn: 0.0115418	total: 1.76s	remaining: 1.29s
519:	learn: 0.011

### Catboost Regressor performs better.

In [136]:
model = catBModel

In [118]:
y_test_predict = np.exp(model.predict(X_test))
y_train_predict = np.exp(model.predict(X_train))


In [119]:
sampledata = pd.read_csv('/content/sample_submission.csv')

In [120]:
df = pd.concat([pd.DataFrame(y_id),pd.DataFrame(y_test_predict)], axis=1, ignore_index=True)
df.columns = ['Id','SalePrice']


In [121]:
df.to_csv('Submission.csv', index=False)
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f /content/Submission.csv -m "Catboost Classifier"

100% 33.6k/33.6k [00:02<00:00, 16.6kB/s]
Successfully submitted to House Prices: Advanced Regression Techniques